In [6]:
# Fill in Lidar DEM no data pixels with USGS/NOAA DEM 

# Import necessary packages  
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np


# Resample USGS/NOAA image from coarse resolution to 1m  
def resample_image(src_image, match_image):
    """Resample src_image to match the resolution of match_image."""
    transform, width, height = calculate_default_transform(
        match_image.crs, src_image.crs, src_image.width, src_image.height, *src_image.bounds)
    kwargs = match_image.meta.copy()
    kwargs.update({
        'crs': src_image.crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    
    data = np.empty(shape=(match_image.count, height, width), dtype=match_image.dtypes[0])
    for i in range(1, match_image.count + 1):
        reproject(
            source=rasterio.band(match_image, i),
            destination=data[i-1],
            src_transform=match_image.transform,
            src_crs=match_image.crs,
            dst_transform=transform,
            dst_crs=src_image.crs,
            resampling=Resampling.nearest)
    
    return data, kwargs

# Overlay lidar DEM and USGS/NOAA DEM 
def overlay_images(top_image_path, bottom_image_path, output_path):
    with rasterio.open(top_image_path) as top_image, rasterio.open(bottom_image_path) as bottom_image:
        if top_image.shape != bottom_image.shape:
            bottom_data, bottom_meta = resample_image(top_image, bottom_image)
        else:
            bottom_data = bottom_image.read()
            bottom_meta = bottom_image.meta.copy()
        
        top_data = top_image.read()
        
        # Assumption: NoData value is the same for all bands
        nodata_value = top_image.nodata if top_image.nodata is not None else 0
        
        # Overlay images
        result_data = np.where(top_data == nodata_value, bottom_data, top_data)
        
        # Update metadata for the output image
        bottom_meta.update({
            'driver': 'GTiff',
            'height': top_image.height,
            'width': top_image.width,
            'transform': top_image.transform,
            'crs': top_image.crs,
            'count': top_image.count
        })
        
        with rasterio.open(output_path, 'w', **bottom_meta) as dest:
            dest.write(result_data)

# Define file paths
top_image_path = "your\path\to\LiDAR\DEM\file"
bottom_image_path = "your\path\to\NOAA\DEM\file"
output_path = "your\output\path"

# Overlay the images
overlay_images(top_image_path, bottom_image_path, output_path)